# Project:  Subreddit Post Analysis: r/iosgaming, r/android gaming

### Logistic Regression Modeling
We will test our data imported from `subs.csv`. The data will be cleaned and prepared for GridSearching across various hyperparameters selected for our features. <br><br> Subreddit submission posts alongside accompanying post contents have been scraped. Target variable `y` is whether or not the document/row belongs to r/ios_gaming or not. `1` - indicates yes, `0` - indicates no (belongs to r/androidgaming instead)
**Contents:**
- [Library Imports](#Library-Imports)
- [Data Import and Cleaning](#Data-Import-and-Cleaning)
- [Modeling Hyperparameters](#Modeling-Hyperparameters)
 - [1. Count Vectorizer](#1.-Count-Vectorizer)
 - [2. TfidfVectorizer](#2.-TfidfVectorizer)
- [Scoring](#Scoring)

### Library Imports

In [1]:
# import everything, we'll be using everything if given the tim
import pandas as pd
import numpy as np
import requests
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
import random
import string

### Data Import and Cleaning

In [3]:
subsdf = pd.read_csv('../datasets/subs.csv', )
subsdf.drop(columns='Unnamed: 0', inplace = True)
subsdf['corpus'] = subsdf['title'] + subsdf['selftext']

In [4]:
subsdf['is_iosgaming'].value_counts()

1    10700
0    10700
Name: is_iosgaming, dtype: int64

### Modeling Hyperparameters

#### 1. Count Vectorizer
- `CountVectorizer()`
- `LogisticRegression()`
- `Pipeline()`
- `GridSearchCV()`

In [5]:
X = subsdf['corpus']  #needs to be in one column
y = subsdf['is_iosgaming']

In [7]:
punct = [i for i in string.punctuation]
stops = stopwords.words('english')
stops.extend(punct)

In [8]:
# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 14) 

In [9]:
#Logistic Regression, CountVectorizer HyperParameters
#pipline
pipe = Pipeline([
    ('cvec', CountVectorizer(lowercase = True)),
    ('lr', LogisticRegression(solver = 'saga', max_iter = 10_000, penalty = 'none' ))
])

#params
pipe_params = {
    'cvec__max_features' : [100, 500, 1000],
    'cvec__ngram_range' : [(1,1), (1,2), (2,2)],
    'cvec__stop_words' : [None, 'english', stops]#, 
#     'cvec__max_df' : (.99, .95, .9) # test different high used words
}

#gridsearch
gs = GridSearchCV(pipe,
                 param_grid=pipe_params,
                 cv = 5)

In [10]:
gs.fit(X_train, y_train);

In [11]:
# Our model's scoring and best params
print(f'Model Training Score: {gs.best_score_}')
print(f'Model Test Score: {gs.score(X_test, y_test)}')
print(gs.best_params_)
print(gs.best_estimator_)
best_lrcv = gs.best_estimator_

Model Training Score: 0.9936448598130841
Model Test Score: 0.9958878504672897
{'cvec__max_features': 500, 'cvec__ngram_range': (1, 1), 'cvec__stop_words': ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'whe

#### 2. TfidfVectorizer
- `TfidfVectorizer()`
- `LogisticRegression()`
- `Pipeline()`
- `GridSearchCV()`

In [ ]:
# establish our target and features
X = subsdf['corpus']
y = subsdf['is_iosgaming']

In [ ]:
# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 14) 

In [15]:
#Logistic Regression, TfidfVectorizer HyperParameters
#pipline
pipe = Pipeline([
    ('tvec', TfidfVectorizer(lowercase = True)),
    ('lr', LogisticRegression(solver = 'saga', max_iter = 10_000, penalty = 'none' ))
])

#params
pipe_params = {
    'tvec__max_features' : [100, 500, 1000],
    'tvec__ngram_range' : [(1,1), (1,2), (2,2)],
    'tvec__stop_words' : [None, 'english', stops]#, 
#     'tvec__max_df' : (.99, .95, .9) # test different high used words
}

#gridsearch
gs = GridSearchCV(pipe,
                 param_grid=pipe_params,
                 cv = 5)

In [16]:
gs.fit(X_train, y_train);

In [17]:
# Our model's scoring and best params
print(f'Model Training Score: {gs.best_score_}')
print(f'Model Test Score: {gs.score(X_test, y_test)}')
print(gs.best_params_)
print(gs.best_estimator_)
best_lrtv = gs.best_estimator_

Model Training Score: 0.9954517133956386
Model Test Score: 0.9962616822429906
{'tvec__max_features': 1000, 'tvec__ngram_range': (1, 1), 'tvec__stop_words': None}
Pipeline(memory=None,
         steps=[('tvec',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=1000,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
          

### Scoring
- `Pipe`: `LogisticRegression()`, `CountVectorizer()`
- `Best Score:` 0.9936448598130841
- `Test Score:` 0.9958878504672897
<br><br>
- `Pipe`: `LogisticRegression()`, `TfidfVectorizer()`
- `Best Score:` 0.9954517133956386
- `Test Score:` 0.9962616822429906
